In [ ]:
 from google.colab import drive
 drive.mount('/content/drive')


In [2]:
import pandas as pd
import time

path='/content/drive/MyDrive/Colab Notebooks//Project/creditcard.csv'
df = pd.read_csv(path)
start_time_now = time.time()


KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import sklearn

from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.linear_model import Ridge, Lasso, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.ensemble import AdaBoostClassifier

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df.head()

In [ ]:
 df['Class'].value_counts()

In [ ]:
print(df.groupby('Class')['Class'].count()/df['Class'].count()*100)
(df.groupby('Class')['Class'].count()/df['Class'].count()*100).plot.pie()

In [ ]:
classes=df['Class'].value_counts()
nomral_share=classes[0]/df['Class'].count()*100
fraud_share=classes[1]/df['Class'].count()*100

In [ ]:
#plt.figure(figsize=(7,5))
#sns.countplot(df['Class'])
#plt.show()

In [ ]:
#FIND THE CORELATION
#corr
corr=df.corr()

In [ ]:
plt.figure(figsize=(24,18))

sns.heatmap(corr,cmap='coolwarm', annot=True)
plt.show()

Find the distribution

In [ ]:
Delta_Time = pd.to_timedelta(df['Time'], unit='s')

df['Time_Day']=(Delta_Time.dt.components.days).astype(int)
df['Time_Hour']=(Delta_Time.dt.components.hours).astype(int)
df['Time_Min']=(Delta_Time.dt.components.minutes).astype(int)


In [ ]:
df.drop('Time',axis=1, inplace=True)


In [ ]:

df.drop(['Time_Day','Time_Min'],axis=1, inplace=True)


In [ ]:
not_frauds=df.query('Class == 0')
frauds=df.query('Class == 1')
balanced_df= pd.concat([frauds,not_frauds.sample(len(frauds)+100000,random_state=40)])

In [ ]:
y=balanced_df['Class']
X=balanced_df.drop('Class',axis=1)


In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X,y, random_state=100, test_size=0.2)

In [ ]:
cols= list(X.columns.values)
cols

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from re import sub

normal_records = df.Class == 0
fraud_records = df.Class == 1

plt.figure(figsize=(20, 60))
for n, col in enumerate(cols):
    plt.subplot(10, 3, n+1)
    sns.distplot(X[col][normal_records], color='green')
    sns.distplot(X[col][fraud_records], color='red')
    plt.title(col, fontsize=17)

    # Set legend
    plt.legend(labels=['Normal', 'Fraud'])

plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define flierprops
flierprops = dict(marker='x', markerfacecolor='red', markersize=6, linestyle='none')

# Iterate over the range
for i in range(10):
    sns.boxplot(x=df['Class'], y=df.columns[i], data=df, flierprops=flierprops)
    plt.title(f'Boxplot of {df.columns[i]} by y')
    plt.xlabel('y')
    plt.ylabel(df.columns[i])
    plt.show()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

pca_2 = PCA(n_components=2)
X_train_pca_2_fitted = pca_2.fit_transform(X_train_scaled)
X_train_pca_2 = pca_2.transform(X_train_scaled)

plt.figure(figsize=(8, 6))
plt.scatter(X_train_pca_2[:, 0], X_train_pca_2[:, 1], marker='x', c=y_train, cmap='viridis', alpha=0.5)
plt.title('Feature Space with 2 Principal Components')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar(label='Class (y)')
plt.grid(True)
plt.show()

In [ ]:
df_pca_2 = pd.DataFrame(X_train_pca_2, columns=['PC1', 'PC2'])
df_pca_2['y'] = y_train.values

plt.figure(figsize=(4, 6))

plt.subplot(1, 2, 1)
sns.boxplot(x='y', y='PC1', data=df_pca_2, flierprops={"marker": "x"})
plt.title('PC1 by Class')

plt.subplot(1, 2, 2)
sns.boxplot(x='y', y='PC2', data=df_pca_2, flierprops={"marker": "x"})
plt.title('PC2 by Class')

plt.tight_layout()
plt.show()


In [ ]:
loadings = pca_2.components_.T
num_features = loadings.shape[0]

plt.figure(figsize=(12, 8))
for i in range(num_features):
    plt.arrow(0, 0, loadings[i, 0], loadings[i, 1], color='r', alpha=0.075)
    plt.text(loadings[i, 0] * 1.15, loadings[i, 1] * 1.15, X_train.columns[i], color='g', ha='center', va='center')

plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Loadings for PC1 and PC2')
plt.axhline(0, color='b', linestyle='--')
plt.axvline(0, color='b', linestyle='--')
plt.grid(True)
plt.show()


In [ ]:
pca = PCA() #Principal component analysis
X_train_pca = pca.fit_transform(X_train_scaled)

variance = pca.explained_variance_ratio_

cumulative_variance = variance.cumsum()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='-')
plt.title('Cumulative Variance Explained by Each Principal Component')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.axhline(y=0.95, color='r', linestyle='--', label='95% Explained Variance')
plt.axhline(y=0.99, color='g', linestyle='--', label='99% Explained Variance')
plt.legend()
plt.show()


#Model Building


In [ ]:
df_Results= pd.DataFrame(columns=['Methodology','Model','Accuracy','roc_value','threshold'])

def Plot_confusion_matrix(y_test,pred_test):
  cm=confusion_matrix(y_test,pred_test)
  plt.clf()
  plt.imshow(cm,interpolation='nearest',cmap=plt.cm.Accent)
  categoryNames=['Not-Fraudalent','Fraudalent']
  plt.title('')

  plt.title('Confusion Matrix- Test Data')
  plt.ylabel("True Label")
  plt.xlabel("Predicted label")
  ticks = np. arange(len(categoryNames) )
  plt.xticks(ticks, categoryNames, rotation=45)
  plt.yticks(ticks, categoryNames)
  s = [['TN',"FP"],['FN',"TP"]]


  for i in range(2):
    for j in range(2):
      plt.text(j,i,str(s[i][j])+'='+str(cm[i][j]),fontsize=12)
  plt.show()


##LogisticModels

In [ ]:

def buildAndRunLogisticModels(df_Results,Methodology, X_train, y_train, X_test, y_test):

  from sklearn import linear_model
  from sklearn.model_selection import KFold

  num_C = list(np.power(10.0, np.arange(-10,10)))
  cv_num = KFold(n_splits=10, shuffle=True, random_state=42)

  searchCV_l2 = linear_model.LogisticRegressionCV(Cs=num_C, penalty='l2',
                                                  scoring='roc_auc',cv=cv_num,random_state=42,max_iter=10000,
                                                  fit_intercept=True,solver='newton-cg',tol=10)

  searchCV_l1 = linear_model.LogisticRegressionCV(Cs=num_C, penalty='l1',
                                                  scoring='roc_auc',cv=cv_num,random_state=42,max_iter=10000,
                                                  fit_intercept=True,solver='liblinear',tol=10)



  searchCV_l1.fit(X_train,y_train)
  searchCV_l2.fit(X_train, y_train)


  print ('Max auc_roc for l1:', searchCV_l1.scores_[1].mean (axis=0).max())
  print ('Max auc_roc for l2:', searchCV_l2.scores_[1].mean (axis=0).max())
  print("Parameters for l1 regularisations")
  print(searchCV_l1.coef_)
  print(searchCV_l1. intercept_)
  print(searchCV_l1. scores_)

  print("Parameters for l2 regularisations")
  print(searchCV_l2. coef_)
  print(searchCV_l2. intercept_)
  print (searchCV_l2. scores_)
  #find predicted vallues
  y_pred_l1 = searchCV_l1.predict(X_test)
  y_pred_l2 = searchCV_l2.predict(X_test)


  #Find predicted probabilities
  y_pred_probs_l1 = searchCV_l1.predict_proba(X_test)[:,1]
  y_pred_probs_l2 = searchCV_l2.predict_proba(X_test)[:,1]

  Accuracy_l2 = metrics.accuracy_score(y_pred=y_pred_l2, y_true=y_test)
  Accuracy_l1 = metrics.accuracy_score(y_pred=y_pred_l1, y_true=y_test)

  print("Accuarcy of Logistic model with l2 regulaription: {0}". format(Accuracy_l2))
  print("Confusion Matrix")
  Plot_confusion_matrix(y_test, y_pred_l2)
  print("classification Report")
  print(classification_report(y_test, y_pred_l2))

  print("Accuarcy of Logistic model with l1 regularisation : {0}".format(Accuracy_l1))
  print("Confusion Matrix")
  Plot_confusion_matrix(y_test, y_pred_l1)
  print("classification Report")
  print(classification_report(y_test, y_pred_l1))

  l2_roc_value = roc_auc_score(y_test, y_pred_probs_l2)
  print('l2 roc_value: {0}'.format(l2_roc_value))
  fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_probs_l2)
  threshold = thresholds[np.argmax(tpr-fpr)]
  print("l2 threshold: {0}".format(threshold))

  roc_auc = metrics.auc(fpr, tpr)
  print("ROC for the test dataset", '{:.1%}'.format(roc_auc))
  plt.plot(fpr,tpr,label="Test, auc="+str(roc_auc))
  plt.legend(loc=4)
  plt.show()

  new_row =   pd.DataFrame({'Methodology':Methodology,
                                               'Model':'Logistic Regression with L2 Regularisation',
                                               'Accuracy': Accuracy_l2, 'roc_value': l2_roc_value,
                                               'threshold': threshold},index=[0])

  df_Results = pd.concat([df_Results, new_row], ignore_index=True)


  l1_roc_value = roc_auc_score(y_test, y_pred_probs_l1)
  print("l1 roc_value:{0}".format(l1_roc_value))
  fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_probs_l1)
  threshold = thresholds[np.argmax(tpr-fpr)]
  print("l1 threshold: {0}".format(threshold))

  roc_auc = metrics.auc(fpr, tpr)
  print("ROC for the test dataset", '{:.1%}'.format(roc_auc))
  plt.plot(fpr,tpr, label="Test, auc="+str(roc_auc))
  plt.legend(loc=4)
  plt.show()




  new_row =  pd.DataFrame({'Methodology':Methodology,
                                               'Model':'Logistic Regression with L1 Regularisation',
                                               'Accuracy': Accuracy_l1, 'roc_value': l1_roc_value,
                                               'threshold': threshold},index=[0])

  df_Results = pd.concat([df_Results, new_row], ignore_index=True)


  return df_Results


##KNNModels

In [ ]:
df_Results=pd.DataFrame()

In [ ]:
def buildAndRunKNNModels(df_Results,Methodology, X_train, y_train, X_test, y_test):


  knn = KNeighborsClassifier(n_neighbors=5, n_jobs=16)
  knn.fit(X_train,y_train)
  score=knn.score(X_test,y_test)
  print('model score',score)

  y_pred=knn.predict(X_test)
  knn_accuracy=metrics.accuracy_score(y_pred=y_pred, y_true=y_test)
  #print(y_pred)
  print("confusion_matrix")
  Plot_confusion_matrix(y_test, y_pred)
  print("classification Report")
  print(classification_report(y_test, y_pred))



  knn_probs = knn.predict_proba(X_test)[:, 1]

  # Calculate roc auc
  knn_roc_value = roc_auc_score(y_test, knn_probs)
  print("KNN roc_value: {0}" .format(knn_roc_value))
  fpr, tpr, thresholds = metrics.roc_curve(y_test, knn_probs)
  threshold = thresholds[np.argmax(tpr-fpr)]
  print ("KNN threshold: {0}",format(threshold))

  roc_auc = metrics.auc(fpr, tpr)
  print('ROC for the test dataset','{:.1%}'.format(roc_auc))
  plt.plot(fpr, tpr, label="Test, auc="+str(roc_auc))
  plt.legend(loc=4)
  plt.show()



  new_row = pd.DataFrame({'Methodology': Methodology,
                            'Model': 'KNN Model',
                            'Accuracy': score,
                            'roc_value': knn_roc_value,
                            'threshold': threshold}, index=[0])

  df_Results = pd.concat([df_Results, new_row], ignore_index=True)
  return df_Results


##TreeModels

In [ ]:
def buildAndRunTreeModels(df_Results, Methodology, X_train,y_train, X_test, y_test ):
#Evaluate Decision Tree model with 'gini' & 'entropy'
  criteria = ['gini', 'entropy']
  scores = {}


  for c in criteria:
    dt = DecisionTreeClassifier (criterion = c, random_state=42)
    dt.fit(X_train, y_train)
    y_pred = dt.predict(X_test)
    test_score = dt.score(X_test, y_test)
    tree_preds = dt.predict_proba(X_test)[:, 1]
    tree_roc_value = roc_auc_score(y_test, tree_preds)
    scores = test_score
    print(c + " score: {0}".format(test_score))
    print("Confusion Matrix")
    Plot_confusion_matrix(y_test, y_pred)
    print("classification Report")
    print(classification_report(y_test, y_pred))
    print(c + " tree_roc_value: {0}" , format(tree_roc_value))
    fpr, tpr, thresholds = metrics.roc_curve(y_test, tree_preds)
    threshold = thresholds[np.argmax(tpr-fpr)]
    print("Tree threshold: {0}".format(threshold))
    roc_auc = metrics.auc(fpr, tpr)
    print("ROC for the test dataset",'{:.1%}'.format(roc_auc))
    plt.plot(fpr, tpr, label="Test, auc="+str(roc_auc))
    plt.legend (loc=4)
    plt.show()



    new_row =  pd.DataFrame({'Methodology':Methodology,
                                               'Model':'Tree Model with {0}criteria'.format(c),
                                               'Accuracy': scores, 'roc_value': tree_roc_value,
                                               'threshold': threshold},index=[0])



    df_Results = pd.concat([df_Results, new_row], ignore_index=True)

    return df_Results

##RandomForestModels

In [ ]:
def buildAndRunRandomForestModels(df_Results, Methodology, X_train,y_train, X_test, y_test ):
#Evaluate Random Forest model
# Create the model with 100 trees

  RF_model = RandomForestClassifier(n_estimators=100,
  bootstrap = True,
  max_features = 'sqrt', random_state=42)

  # Fit on training data
  RF_model.fit(X_train, y_train)
  RF_test_score = RF_model.score(X_test, y_test)
  RF_model.predict(X_test)

  print('Model Accuracy: {0}'. format(RF_test_score))

  # Actual class predictions
  rf_predictions = RF_model.predict(X_test)

  print ("Confusion Matrix")
  Plot_confusion_matrix(y_test, rf_predictions)
  print("classification Report")
  print(classification_report(y_test, rf_predictions))

  # Probabilities for each class
  rf_probs = RF_model.predict_proba(X_test)[:, 1]

  # Calculate roc auc
  roc_value = roc_auc_score(y_test, rf_probs)


  print ("Random Forest roc_value: {0}" .format(roc_value))
  fpr, tpr, thresholds = metrics.roc_curve(y_test, rf_probs)
  threshold = thresholds[np.argmax(tpr-fpr)]
  print ("Random Forest threshold: {0}". format(threshold))
  roc_auc = metrics.auc(fpr, tpr)
  print("ROC for the test dataset", '{:.1%}'.format(roc_auc))
  plt.plot(fpr, tpr,label="Test, auc="+str(roc_auc))
  plt.legend(loc=4)
  plt.show()


  new_row =  pd.DataFrame({'Methodology':Methodology,
                                               'Model':'Random Forest',
                                               'Accuracy': RF_test_score, 'roc_value': roc_value,
                                               'threshold': threshold},index=[0])
  df_Results = pd.concat([df_Results, new_row], ignore_index=True)

  return df_Results


##XGBoostModels

In [ ]:
def buildAndRunXGBoostModels(df_Results, Methodology,X_train,y_train, X_test, y_test ):
#EvaLuate XGboost moded
  XGBmodel = XGBClassifier(random_state=42)
  XGBmodel.fit(X_train, y_train)
  y_pred = XGBmodel.predict(X_test)

  XGB_test_score = XGBmodel.score(X_test, y_test)
  print('Model Accuracy: {0}'.format(XGB_test_score))

  print("Confusion Matrix")
  Plot_confusion_matrix(y_test, y_pred)
  print("classification Report")
  print(classification_report(y_test, y_pred))
  # Probabilities for each class
  XGB_probs = XGBmodel.predict_proba(X_test)[:, 1]
  # Calculate roc auc
  XGB_roc_value = roc_auc_score(y_test, XGB_probs)

  print("Xboost roc_value: {0}".format(XGB_roc_value) )
  fpr, tpr, thresholds = metrics.roc_curve(y_test, XGB_probs)
  threshold = thresholds[np.argmax(tpr-fpr)]
  print("XBoost threshold: {0}". format(threshold))
  roc_auc = metrics.auc(fpr, tpr)
  print("ROC for the test dataset", '{:.1%}'.format(roc_auc))
  plt.plot(fpr,tpr,label="Test, auc="+str(roc_auc))
  plt.legend(loc=4)
  plt.show( )

  new_row =  pd.DataFrame({'Methodology':Methodology,
                                               'Model':'XGB boost Model',
                                               'Accuracy': XGB_test_score, 'roc_value': XGB_roc_value,
                                               'threshold': threshold},index=[0])
  df_Results = pd.concat([df_Results, new_row], ignore_index=True)


  return df_Results


##SVMModels

In [ ]:
def buildAndRunSVMModels (df_Results, Methodology, X_train,y_train,X_test, y_test):

#Evaluate SVM model with sigmoid kernel model
  from sklearn.svm import SVC
  from sklearn.metrics import accuracy_score
  from sklearn.metrics import roc_auc_score

  clf = SVC(kernel='sigmoid', random_state=42)
  clf.fit (X_train,y_train)
  y_pred_SVM = clf.predict(X_test)
  SVM_Score = accuracy_score(y_test,y_pred_SVM)
  print("accuracy_score: {0}".format(SVM_Score))
  print("Confusion Matrix")
  Plot_confusion_matrix(y_test, y_pred_SVM)
  print("classification Report")
  print(classification_report(y_test, y_pred_SVM))

  # Run classifier
  classifier = SVC(kernel='sigmoid' , probability=True)
  svm_probs = classifier.fit(X_train, y_train).predict_proba(X_test)[:, 1]

  # Calculate roc auc
  roc_value = roc_auc_score(y_test, svm_probs)

  print("SVM roc_value: {0}".format(roc_value))
  fpr, tpr, thresholds = metrics.roc_curve(y_test, svm_probs)
  threshold = thresholds[np.argmax(tpr-fpr)]
  print("SVM threshold: {0}".format(threshold))
  roc_auc = metrics.auc(fpr, tpr)
  print("ROC for the test dataset", '{:.1%}'.format(roc_auc))
  plt.plot(fpr, tpr, label="Test, auc="+str(roc_auc))
  plt. legend(loc=4)
  plt.show()


  new_row =  pd.DataFrame({'Methodology':Methodology,
                                               'Model':'SVM Model',
                                               'Accuracy': SVM_Score, 'roc_value': roc_value,
                                               'threshold': threshold},index=[0])
  df_Results = pd.concat([df_Results, new_row], ignore_index=True)

  return df_Results

#1.Perform Cross validation with RepeatedKFold

In [ ]:
from sklearn.model_selection import RepeatedKFold
rkf= RepeatedKFold(n_splits=5, n_repeats=10, random_state=None)

for train_index, test_index in rkf.split(X,y):
  print('Train:', train_index, 'Test:', test_index)
  X_train_cv, X_test_cv = X.iloc[train_index], X.iloc[test_index]
  y_train_cv, y_test_cv = y.iloc[train_index], y.iloc[test_index]

In [ ]:
print ("Logistic Regression with L1 And L2 Regularisation")
start_time = time.time()
df_Results = buildAndRunLogisticModels(df_Results, "RepeatedKFold Cross Validation", X_train_cv,y_train_cv, X_test_cv, y_test_cv)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print('-'*60 )


In [ ]:

#Run KNN Model
print ("KNN Model")
start_time = time.time()
df_Results = buildAndRunKNNModels(df_Results, "RepeatedKFold Cross Validation", X_train_cv,y_train_cv, X_test_cv, y_test_cv)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:

#Run Decision Tree Models with 'gini' & 'entropy' criteria
print("Decision Tree Models with 'gini' & 'entropy' criteria")
start_time = time.time()
df_Results = buildAndRunTreeModels(df_Results, "RepeatedKFold Cross Validation", X_train_cv,y_train_cv, X_test_cv, y_test_cv)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:

#Run RF Model
print ("RF Model")
start_time = time.time()
df_Results = buildAndRunRandomForestModels(df_Results, "RepeatedKFold Cross Validation", X_train_cv,y_train_cv, X_test_cv, y_test_cv)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:

#Run XGB Model
print ("XGB Model")
start_time = time.time()
df_Results = buildAndRunXGBoostModels(df_Results, "RepeatedKFold Cross Validation", X_train_cv,y_train_cv, X_test_cv, y_test_cv)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:
#Run SVM Model
print ("SVM Model")
start_time = time.time()
df_Results = buildAndRunSVMModels(df_Results, "RepeatedKFold Cross Validation", X_train_cv,y_train_cv, X_test_cv, y_test_cv)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


#2.Perform Cross validation with StratifiedKFold

In [ ]:
from sklearn.model_selection import StratifiedKFold
rkf= StratifiedKFold(n_splits=5, random_state=None)

for train_index, test_index in rkf.split(X,y):
  print('Train:', train_index, 'Test:', test_index)
  X_train_SKF_cv, X_test_SKF_cv = X.iloc[train_index], X.iloc[test_index]
  y_train_SKF_cv, y_test_SKF_cv = y.iloc[train_index], y.iloc[test_index]

In [ ]:
print ("Logistic Regression with L1 And L2 Regularisation")
start_time = time.time()
df_Results = buildAndRunLogisticModels(df_Results, "StratifiedKFold Cross Validation", X_train_SKF_cv,y_train_SKF_cv, X_test_SKF_cv, y_test_SKF_cv)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print('-'*60 )


In [ ]:

#Run Decision Tree Models with 'gini' & 'entropy' criteria
print("Decision Tree Models with 'gini' & 'entropy' criteria")
start_time = time.time()
df_Results = buildAndRunTreeModels(df_Results, "StratifiedKFold Cross Validation", X_train_SKF_cv,y_train_SKF_cv, X_test_SKF_cv, y_test_SKF_cv)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )



In [ ]:

#Run Decision Tree Models with 'gini' & 'entropy' criteria
print("Decision Tree Models with 'gini' & 'entropy' criteria")
start_time = time.time()
df_Results = buildAndRunTreeModels(df_Results, "StratifiedKFold Cross Validation", X_train_SKF_cv,y_train_SKF_cv, X_test_SKF_cv, y_test_SKF_cv)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:

#Run RF Model
print ("RF Model")
start_time = time.time()
df_Results = buildAndRunRandomForestModels(df_Results, "StratifiedKFold Cross Validation", X_train_SKF_cv,y_train_SKF_cv, X_test_SKF_cv, y_test_SKF_cv)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:

#Run XGB Model
print ("XGB Model")
start_time = time.time()
df_Results = buildAndRunXGBoostModels(df_Results, "StratifiedKFold Cross Validation", X_train_SKF_cv,y_train_SKF_cv, X_test_SKF_cv, y_test_SKF_cv)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:

#Run SVM Model
print ("SVM Model")
start_time = time.time()
df_Results = buildAndRunSVMModels(df_Results, "StratifiedKFold Cross Validation", X_train_SKF_cv,y_train_SKF_cv, X_test_SKF_cv, y_test_SKF_cv)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


#Model building with balancing Classes

##3.Oversampling with RandomOverSampler with StratifiedKFold Cross Validation

In [ ]:
X=X.values
y=y.values

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:

ros = RandomOverSampler()
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    ROS = RandomOverSampler(sampling_strategy=0.5)
    X_over, y_over = ros.fit_resample(X_train, y_train)

    # Proceed with model training and evaluation using the oversampled data

In [ ]:
Data_Imbalance_Handiling = 'Random Oversampling with StratifiedKFold CV'

In [ ]:

start_time = time.time()
df_Results = buildAndRunLogisticModels(df_Results, Data_Imbalance_Handiling, X_over,y_over, X_test, y_test)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:

#Run Decision Tree Models with 'gini' & 'entropy' criteria
print("Decision Tree Models with 'gini' & 'entropy' criteria")
start_time = time.time()
df_Results = buildAndRunKNNModels(df_Results, Data_Imbalance_Handiling, X_over,y_over, X_test, y_test)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:

#Run Decision Tree Models with 'gini' & 'entropy' criteria
print("Decision Tree Models with 'gini' & 'entropy' criteria")
start_time = time.time()
df_Results = buildAndRunTreeModels(df_Results, Data_Imbalance_Handiling, X_over,y_over, X_test, y_test)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:

#Run RF Model
print ("RF Model")
start_time = time.time()
df_Results = buildAndRunRandomForestModels(df_Results, Data_Imbalance_Handiling, X_over,y_over, X_test, y_test)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:

#Run Decision Tree Models with 'gini' & 'entropy' criteria
print("Decision Tree Models with 'gini' & 'entropy' criteria")
start_time = time.time()
df_Results = buildAndRunXGBoostModels(df_Results, Data_Imbalance_Handiling, X_over,y_over, X_test, y_test)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:
#Run Decision Tree Models with 'gini' & 'entropy' criteria
print("Decision Tree Models with 'gini' & 'entropy' criteria")
start_time = time.time()
df_Results = buildAndRunSVMModels(df_Results, Data_Imbalance_Handiling, X_over,y_over, X_test, y_test)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


##Oversampling with SMOTE Oversampling

> Add blockquote



In [ ]:
from sklearn.model_selection import StratifiedKFold
from imblearn import over_sampling

skf = StratifiedKFold(n_splits=5, random_state=None)

for fold, (train_index, test_index) in enumerate(skf.split(X,y), 1):

  X_train = X[train_index]
  y_train = y[train_index]
  X_test = X[test_index]
  y_test = y[test_index]
  SMOTE = over_sampling.SMOTE(random_state=0)
  X_train_Smote, y_train_Smote= SMOTE.fit_resample(X_train, y_train)

X_train_Smote = pd.DataFrame(data=X_train_Smote,columns=cols)

In [ ]:
Data_Imbalance_Handiling='Smote Oversampling with StratifiedKFold CV'

In [ ]:

start_time = time.time()
df_Results = buildAndRunLogisticModels(df_Results, Data_Imbalance_Handiling, X_train_Smote,y_train_Smote, X_test, y_test)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:

start_time = time.time()
df_Results = buildAndRunKNNModels(df_Results, Data_Imbalance_Handiling, X_train_Smote,y_train_Smote, X_test, y_test)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:

start_time = time.time()
df_Results = buildAndRunTreeModels(df_Results, Data_Imbalance_Handiling, X_train_Smote,y_train_Smote, X_test, y_test)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:

#Run RF Model
print ("RF Model")
start_time = time.time()
df_Results = buildAndRunRandomForestModels(df_Results, Data_Imbalance_Handiling, X_train_Smote,y_train_Smote, X_test, y_test)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:

start_time = time.time()
df_Results = buildAndRunXGBoostModels(df_Results, Data_Imbalance_Handiling, X_train_Smote,y_train_Smote, X_test, y_test)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:

start_time = time.time()
df_Results = buildAndRunSVMModels(df_Results, Data_Imbalance_Handiling, X_train_Smote,y_train_Smote, X_test, y_test)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


##Oversampling with ADASYN Oversampling

In [ ]:
from sklearn.model_selection import StratifiedKFold
from imblearn import over_sampling

skf = StratifiedKFold(n_splits=5, random_state=None)

for fold, (train_index, test_index) in enumerate(skf.split(X,y), 1):

  X_train = X[train_index]
  y_train = y[train_index]
  X_test = X[test_index]
  y_test = y[test_index]
  ADASYN = over_sampling.ADASYN(random_state=0)
  X_train_ADASYN, y_train_ADASYN= ADASYN.fit_resample(X_train, y_train)

X_train_ADASYN = pd.DataFrame(data=X_train_Smote,columns=cols)

In [ ]:
from imblearn.over_sampling import ADASYN
ada = ADASYN(random_state=42)
X_train_ADASYN, y_train_ADASYN = ada.fit_resample(X, y)
X_over.shape,y_over.shape

In [ ]:
Data_Imbalance_Handiling='ADASYN Oversampling with StratifiedKFold CV'

In [ ]:

start_time = time.time()
df_Results = buildAndRunLogisticModels(df_Results, Data_Imbalance_Handiling, X_train_ADASYN,y_train_ADASYN, X_test, y_test)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:

start_time = time.time()
df_Results = buildAndRunKNNModels(df_Results, Data_Imbalance_Handiling, X_train_ADASYN,y_train_ADASYN, X_test, y_test)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:

start_time = time.time()
df_Results = buildAndRunTreeModels(df_Results, Data_Imbalance_Handiling, X_train_ADASYN,y_train_ADASYN, X_test, y_test)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:

#Run RF Model
print ("RF Model")
start_time = time.time()
df_Results = buildAndRunRandomForestModels(df_Results, Data_Imbalance_Handiling, X_train_ADASYN,y_train_ADASYN, X_test, y_test)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:

start_time = time.time()
df_Results = buildAndRunXGBoostModels(df_Results, Data_Imbalance_Handiling, X_train_ADASYN,y_train_ADASYN, X_test, y_test)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )


In [ ]:
'''
start_time = time.time()
df_Results = buildAndRunSVMModels(df_Results, Data_Imbalance_Handiling, X_train_ADASYN,y_train_ADASYN, X_test, y_test)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )
'''

In [ ]:
df_Results

In [ ]:
df_Results_2=df_Results.copy()

#Demonstrate by specific case

In [ ]:
df_Results.to_csv('df_Results.csv')

In [ ]:
'''
def buildAndRunKNNModels_results(df_Results, Methodology, X_train, y_train, X_test, y_test):

    knn = KNeighborsClassifier(n_neighbors=5, n_jobs=16)
    knn.fit(X_train, y_train)
    score = knn.score(X_test, y_test)
    print('Model score:', score)

    y_pred = knn.predict(X_test)
    knn_accuracy = metrics.accuracy_score(y_pred=y_pred, y_true=y_test)
    print("Confusion matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    knn_probs = knn.predict_proba(X_test)[:, 1]

    # Calculate roc auc
    knn_roc_value = roc_auc_score(y_test, knn_probs)
    print("KNN roc_value:", knn_roc_value)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, knn_probs)
    threshold = thresholds[np.argmax(tpr - fpr)]
    print ("KNN threshold:", threshold)

    roc_auc = metrics.auc(fpr, tpr)
    print('ROC for the test dataset:', '{:.1%}'.format(roc_auc))
    plt.plot(fpr, tpr, label="Test, auc=" + str(roc_auc))
    plt.legend(loc=4)
    plt.show()

    predictions_list = []  # Initialize an empty list for predictions
    predictions_df = pd.DataFrame({'Predicted': y_pred})

    # Append predictions_df to predictions_list
    predictions_list.append(predictions_df)

    # Concatenate all DataFrames in predictions_list
    df_Predictions = pd.concat(predictions_list, ignore_index=True)

    return df_Predictions

df_Predictions=pd.DataFrame()
df_Predictions = buildAndRunKNNModels_results(df_Predictions, 'Methodology', X_train, y_train, X_test, y_test)
print(df_Predictions)
'''

In [ ]:
'''
#Run KNN Model
print ("KNN Model")
start_time = time.time()
df_Predictions = buildAndRunKNNModels_results(df_Predictions, "RepeatedKFold Cross Validation", X_train_cv,y_train_cv, X_test_cv, y_test_cv)
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time))
print ('-'*60 )
'''

In [ ]:
df_Results_2=df_Results.copy()
df_Results_2_filtered = df_Results_2[df_Results_2['threshold'] <= 1.0]
df_Results_2_filtered_sorted = df_Results_2_filtered.sort_values('Accuracy', ascending=False)
df_Results_2_filtered_sorted
df_Results_2_filtered_sorted.to_csv('hello.csv')


In [ ]:
print("Time Taken by Model: --- %s seconds ---"% (time.time() - start_time_now))


In [ ]:
'''
import seaborn as sns
import matplotlib.pyplot as plt

# Define flierprops
flierprops = dict(marker='x', markerfacecolor='red', markersize=6, linestyle='none')

# Iterate over the range
for i in range(10):
    sns.boxplot(x=y_train_cv, y=X_train_cv.columns[i], data=df, flierprops=flierprops)
    plt.title(f'Boxplot of {df.columns[i]} by y')
    plt.xlabel('y')
    plt.ylabel(df.columns[i])
    plt.show()
'''

In [ ]:

def buildAndRunRandomForestModels(df_Results, Methodology, X_train, y_train, X_test, y_test):
    RF_model = RandomForestClassifier(n_estimators=100, bootstrap=True, max_features='sqrt', random_state=42)
    RF_model.fit(X_train, y_train)
    RF_test_score = RF_model.score(X_test, y_test)
    rf_predictions = RF_model.predict(X_test)

    print('Model Accuracy: {0}'.format(RF_test_score))
    print("Confusion Matrix")
    Plot_confusion_matrix(y_test, rf_predictions)
    print("Classification Report")
    print(classification_report(y_test, rf_predictions))

    rf_probs = RF_model.predict_proba(X_test)[:, 1]
    roc_value = roc_auc_score(y_test, rf_probs)

    print("Random Forest roc_value: {0}".format(roc_value))
    fpr, tpr, thresholds = metrics.roc_curve(y_test, rf_probs)
    threshold = thresholds[np.argmax(tpr - fpr)]
    print("Random Forest threshold: {0}".format(threshold))
    roc_auc = metrics.auc(fpr, tpr)
    print("ROC for the test dataset", '{:.1%}'.format(roc_auc))
    plt.plot(fpr, tpr, label="Test, auc=" + str(roc_auc))
    plt.legend(loc=4)
    plt.show()

    # Define empty DataFrame for predictions
    df_predictions = pd.DataFrame()

    # Append predictions to df_predictions DataFrame
    predictions_df = pd.DataFrame({'Predicted': rf_predictions})
    df_predictions = pd.concat([df_predictions, predictions_df], ignore_index=True)

    new_row = pd.DataFrame({'Methodology': Methodology,
                            'Model': 'Random Forest',
                            'Accuracy': RF_test_score, 'roc_value': roc_value,
                            'threshold': threshold}, index=[0])

    df_Results = pd.concat([df_Results, new_row], ignore_index=True)

    return df_predictions


df_predictions = buildAndRunRandomForestModels(df_Results, 'Methodology', X_train_cv, y_train_cv, X_test_cv, y_test_cv)
print(df_predictions)


In [ ]:
df_predictions=df_predictions.squeeze()

In [ ]:
X_test_cv_2=X_test_cv.copy()

In [ ]:
X_test_cv_2['Class']=df_predictions.values
X_test_cv_2

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from re import sub

normal_records = X_test_cv_2.Class == 0
fraud_records = X_test_cv_2.Class == 1

plt.figure(figsize=(20, 60))
for n, col in enumerate(cols):
    plt.subplot(10, 3, n+1)
    sns.distplot(X_test_cv_2[col][normal_records], color='green')
    sns.distplot(X_test_cv_2[col][fraud_records], color='red')
    plt.title(col, fontsize=17)

    # Set legend
    plt.legend(labels=['Normal', 'Fraud'])

plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define flierprops
flierprops = dict(marker='x', markerfacecolor='red', markersize=6, linestyle='none')

# Iterate over the range
for i in range(10):
    sns.boxplot(x=X_test_cv_2['Class'], y=X_test_cv_2.columns[i], data=X_test_cv_2, flierprops=flierprops)
    plt.title(f'Boxplot of {df.columns[i]} by y')
    plt.xlabel('y')
    plt.ylabel(df.columns[i])
    plt.show()


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test_cv)

pca_2 = PCA(n_components=2)
X_test_pca_2_fitted = pca_2.fit_transform(X_test_scaled)
X_test_pca_2 = pca_2.transform(X_test_scaled)

plt.figure(figsize=(8, 6))
plt.scatter(X_test_pca_2[:, 0], X_test_pca_2[:, 1], marker='x', c=df_predictions.values, cmap='viridis', alpha=0.5)
plt.title('Feature Space with 2 Principal Components')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar(label='Class (y)')
plt.grid(True)
plt.show()

In [ ]:
df_pca_2 = pd.DataFrame(X_test_pca_2, columns=['PC1', 'PC2'])
df_pca_2['y'] = df_predictions.values

plt.figure(figsize=(4, 6))

plt.subplot(1, 2, 1)
sns.boxplot(x='y', y='PC1', data=df_pca_2, flierprops={"marker": "x"})
plt.title('PC1 by Class')

plt.subplot(1, 2, 2)
sns.boxplot(x='y', y='PC2', data=df_pca_2, flierprops={"marker": "x"})
plt.title('PC2 by Class')

plt.tight_layout()
plt.show()


In [ ]:
df_Results_2_filtered_sorted